In [3]:
%%js // Exécuter cette cellule pour colorer les cellules des énoncés
for (let cell of Jupyter.notebook.get_cells()) if (cell.metadata.tags && cell.metadata.tags.indexOf('nsi_enonce') >= 0) cell.element.css('background', '#DDDDFF')

<IPython.core.display.Javascript object>

TP
==


Problème du rendu de monnaie
----------------------------

Nous définissons un _système de monnaie_ comme un tableau de $n$ entiers _croissant_ qui liste les valeurs des pièces disponibles. Chaque pièce représente un nombre entier d’unités. La première pièce est toujours 1, pour garantir qu’un paiement est toujours possible. Par exemple, le système des pièces euros est $S_€=(1,2,5,10,20,50,100,200)$ (l’unité est le centime). Si on voulait ajouter le billet de 5 euros, on ajouterait 500 à $S_€$… Un paiement est un tableau $X=(x_1,\dots, x_n)$ indiquant le nombre de pièces de chaque type à utiliser. On suppose que chaque pièce est disponible en quantité illimitée. Le montant du paiement est $\sum_{i=1}^nx_i\times p_i$. Par exemple, $(0,2,0,1,0,0,0,0)$ a pour montant $2\times2+1\times10=14$

Le problème a pour entrées :

-   un système de monnaie $S$ : $p_1=1<\dots<p_n$

-   un montant (entier) à payer $v$

L’objectif est de renvoyer un paiement $X=(x_1,\dots, x_n)$ de montant $v$ qui utilise un nombre minimal de pièces.

En d’autres termes, on cherche $X$ qui _minimise_ $\sum_{i=1}^nx_i$ sous la contrainte $\sum_{i=1}^nx_i\times p_i=v$.

**Question 1 :** Quelle est la solution (optimale) pour $S=S_€$ avec $v=8$ ?

- X = (1, 1, 1, 0, 0, 0, 0, 0)


**Question 2 :** La solution que l’on utilise naturellement pour rendre la monnaie au quotidien est généralement la stratégie gloutonne (sauf lorsque l’on a trop de petites pièces). Écrire un algorithme glouton qui prend en entrée un système, un montant et renvoie la solution gloutonne.

*Indication : parcourir le tableau dans le bon ordre, utiliser la division euclidienne.*


In [2]:
def glouton(s,v):
    n=len(s)            # n --> nombre de pièces différentes (par exemple 1, 2, 5 etc...)
    restant = v
    paiement = [0]*n    # --> crée un tableau avec toutes les pièces rendues (on commence avec 0 pièce de chaque type)

    for i in range(n):  # Pour chaque type de pièce

        montant_piece = s[-i-1]         # --> Montant de ce type de pièce

        if restant >= montant_piece:    # Si le montant restant est superieur ou égal au montant de la pièce actuelle

            paiement[-i-1] += int(restant // montant_piece) # On ajoute un nombre de pièce égal au quotient de la division euclidienne du
                                                            # montant restant par le montant de la pièce
            restant = int(restant % montant_piece)          # Le montant restant est égal au reste de cette division euclidienne

    return paiement

# Test :
print(glouton([1,2,5,10,20,50,100,200],8)==[1, 1, 1, 0, 0, 0, 0, 0])

True


**Question 3 :** Prouver que votre algorithme termine (donner sa complexité) et que la solution renvoyée vaut bien $v$.

- Il n'y a pas de boucle infinie et le montant restant ne peut que réduire donc la boucle va se terminer
- Complexité : O(n)

On dit qu’un système est *canonique* si pour ce système l’algorithme glouton renvoie toujours une solution optimale.

**Question 4 :** On peut montrer (nous ne le ferons pas) que le système $S_€$ est canonique. Montrer que (1,3,4) et (1,5,10,12) ne sont pas canoniques.

- Pour (1, 3, 4) on a le nombre 6 qui donne (2, 0, 1) alors que la solution optimale est (0, 2, 0)
- Pour (1, 5, 10, 12) on a le nombre 15 qui donne (3, 0, 0, 1) alors que la solution optimale est (0, 1, 1, 0)


**Question 5 :** Un système à 3 pièces $(p_1=1,p_2,p_3)$ n’est pas canonique si et seulement si $0<r<p_2-q$ où $r$ est le reste et $q$ le quotient de la division euclidienne de $p_3$ par $p_2$. Écrire une fonction booléenne prenant en entrée un système à 3 pièces et indiquant s’il est canonique.

In [10]:
def canonique3(s:list[int])->bool:
    if len(s) > 3 or len(s) < 3:
        raise ValueError

    q, r = s[2]//s[1], s[2]%s[1]

    if s[0] == 1 and r > 0 and r < (s[1]-q):
        return False
    return True

# Test :
print(canonique3([1,2,5])) # True
print(canonique3([1,3,4])==False)

True
True


Problème de planification
-------------------------

Les algorithmes d’ordonnancement déterminent comment planifier des tâches qui doivent se partager une ressource. C’est-à-dire que ces algorithmes choisissent quelles tâches exécuter, à quel instant, et avec quelles ressources. On utilise ces algorithmes dans les systèmes d’exploitation puisque chaque processeur ne peut exécuter qu’une tâche à la fois, mais aussi dans la vie courante pour planifier la production des usines, concevoir les emplois du temps des lycées et des compagnies de transport, organiser la logistique…

Nous nous intéressons à un problème d’ordonnancement simple dont l’entrée est une liste de tâches $L$. Chaque tâche est une paire de nombres $d_i,f_i$ indiquant l’instant de début $d_i$ et l’instant de fin $f_i$ de la tâche. Deux tâches $i$ et $j$ sont incompatibles lorsque leurs intervalles se chevauchent ($d_i\leq d_j< f_i$ ou $d_j\leq d_i< f_j$). L’objectif est d’effectuer le plus de tâches compatibles possible.

**Question 1 :** Montrer comment l’on peut effectuer 2 tâches pour $L_1=[(0,3),(1,2.5),(2,3),(2.5,4)]$


**Question 2 :** Écrire (a) une fonction `conflit` qui prend en entrée un tableau de $n$ tâches `L`, un tableau de $n$ booléens `B`, l’indice `i` d’une tâche, et qui identifie les tâches de `L` incompatibles avec la tâche `i` (y compris `i`) en mettant l’indice correspondant à `False` dans `B`. La fonction renverra le nombre de valeurs qui viennent de basculer à `False`. Par exemple, en partant de $B=[True,False,True,True]$, après $conflit(L_1,B,1)$ on aura $B=[False,False,False,True]$ et on renverra 2.

Notre algorithme glouton pour calculer un ordonnancement des tâches va maintenir un tableau $B$ indiquant les tâches disponibles (au départ, toutes). A chaque étape, l’algorithme glouton va sélectionner la « meilleure » tâche $t$ encore disponible, puis ajouter aux tâches déjà indisponibles les tâches incompatibles avec $t$, en utilisant `conflit`. Pour définir la meilleure tâche à ajouter à chaque étape parmi les tâches disponibles, on peut envisager au moins 4 critères : (i) la première tâche à débuter, (ii) la première tâche à terminer, (iii) la tâche qui a le moins d’incompatibilités avec les tâches encore disponibles, ou (iv) la tâche la plus courte.


In [ ]:
def conflit(l,b,i):
    """
    Entrée: 
    - tableau de n taches L (une tâche = paire de flottants)
    - tableau de booléens b
    - indice i
    ======
    Modifie (effet de bord) b: indique les tâches incompatibles avec la tâche i
    en mettant à False les indices correspondant dans b
    ======
    Renvoie: le nombre de valeurs modifiées dans b (celles qui étaient à True et deviennent False).
    """
    # A Compléter.

# Test:
L_0=[(1,3),(3,6.5),(3,5),(6,7)]
L_1=[(0,3),(1,2.5),(2,3),(2.5,4)]
B=[True,False,True,True]
print(conflit(L_0,B,1)==2)
print(B==[True, False, False, False])
B=[True,False,True,True]
print(conflit(L_1,B,1)==2)
print(B==[False, False, False, True])

**Question 3 :** Identifier laquelle de ces 4 stratégies pourrait être optimale, en comptant par exemple le nombre de tâches que parvient à effectuer chaque stratégie pour $L_1$ et $L_2$ illustrées ci-dessous.

<embed src="../images/glouton-tp-fig1-nsi.pdf" style="width:128mm" />


**Question 4 :** Implémenter cette stratégie à l’aide du code dans le fichier. On pourra utiliser `append`.

*Remarque :* il est possible de prouver que cette stratégie est optimale (mais nous ne le ferons pas).


In [ ]:
def glouton(l):
    """
    Entrée: une liste de tâches l
    Renvoie: les tâches sélectionnées, sous forme d'une liste de tâches (et non d'un tableau de booléens)
    """
    n = len(l)
    nb_taches_dispos = n
    dispos = [True]*n
    taches = sorted(l,key=lambda t:t[1]) # trier les taches en fonction de leur fin
    taches_selectionnees = []
    while nb_taches_dispos > 0:
        for i in range(n):
            if dispos[i]:
                # A Compléter
    return taches_selectionnees

# Test :
L_1=[(0,3),(1,2.5),(2,3),(2.5,4)]
L_2=[(0,1),(1,2),(2,3),(3,4),
(0.1, 1.1), (0.2, 1.2), (0.3, 1.3), (0.4, 1.4),
(2.9, 3.9), (2.8, 3.8), (2.7, 3.7), (2.6, 3.6),
(1.5,2.5)]
print(glouton(L_0)==[(1, 3), (3, 5), (6, 7)])
print(glouton(L_1)==[(1, 2.5), (2.5, 4)])
print(glouton(L_2)==[(0, 1), (1, 2), (2, 3), (3, 4)])


**Question 5 :** Présenter une stratégie gloutonne symétrique qui calcule l’ordonnancement optimal mais dans l’ordre chronologique inverse.


**Question 6 :** On peut imaginer une autre approche gloutonne. Montrer avec un exemple $L$ à 3 tâches que l’algorithme glouton suivant n’est pas optimal :



In [ ]:
*tantque* Il reste des tâches disponibles *faire*
	*si* elles sont toutes compatibles 
		*alors* on les sélectionne toutes
		*sinon* enlever la plus longue 
	*fin*
*fin*